In [1]:
CUDA_LAUNCH_BLOCKING="1"

In [2]:
# !git clone "https://github.com/anoopkunchukuttan/indic_nlp_library"
# !git clone https://github.com/anoopkunchukuttan/indic_nlp_resources.git

In [3]:
!pip install indic-nlp-library

In [4]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [5]:
import torch
torch.cuda.empty_cache()
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
print(torch.__version__)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

import string
import re
import random
from nltk.tokenize import word_tokenize
from unicodedata import normalize
import numpy as np
from numpy import array

1.8.1+cu101
cuda


In [6]:
from google.colab import drive
drive.mount("/content/gdrive")
import os
import sys
path = '/content/gdrive/My Drive/Dataset'
os.chdir(path)
print(os.getcwd())

Mounted at /content/gdrive
/content/gdrive/My Drive/Dataset


#Tokenizer for hindi language and then using the Devnagri normalizer

In [7]:
from indicnlp.tokenize import indic_tokenize 
# from indicnlp.normalize.indic_normalize import DevanagariNormalizer
# factory = DevanagariNormalizer()

from indicnlp.normalize.indic_normalize import IndicNormalizerFactory
factory=IndicNormalizerFactory()
normalizer=factory.get_normalizer("hi",remove_nuktas=False)

In [8]:
#Lowercase, trim, and remove non-letter characters
def normalizeString(w):
  w = re.sub(r"([?.!,¿-])", r" \1 ", w)
  w = re.sub(r"[().!,?-]+", r" ", w)
  w = re.sub(r'[" "]+', " ", w) #multiple spaces
  w = w.lower().strip() #lowercase #Remove white spaces
  return w

In [9]:
#Using the contractions that can help in decoding

abbr = { 
"ain't": "am not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he's": "he is",
"how'd": "how did",
"how'll": "how will",
"how's": "how is",
"here's": "here is",
"i'd": "i would",
"i'll": "i will",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"it'd": "it would",
"it'll": "it will",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"must've": "must have",
"mustn't": "must not",
"needn't": "need not",
"oughtn't": "ought not",
"shan't": "shall not",
"sha'n't": "shall not",
"she'd": "she would",
"she'll": "she will",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"that'd": "that would",
"that's": "that is",
"there'd": "there had",
"there's": "there is",
"they'd": "they would",
"they'll": "they will",
"they're": "they are",
"they've": "they have",
"wasn't": "was not",
"we'd": "we would",
"we'll": "we will",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"where'd": "where did",
"where's": "where is",
"who'll": "who will",
"who's": "who is",
"won't": "will not",
"wouldn't": "would not",
"you'd": "you would",
"you'll": "you will",
"you're": "you are",
"y'all":"you all",
"let's": "let us"
}
#Dictionary mapping contractions to word

In [10]:
import csv
data = []
with open('train1.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    for row in csvReader:
        data.append([normalizeString(row[0]), normalizeString(row[1])])

for l in data:
  for w in l[1].split():
    if(w in abbr.keys()):
      #print(w)
      l[1] = l[1].replace(w, abbr[w])
      break

data = data[1:]

In [11]:
test = []
with open('hindistatements-2.csv') as csvfile:
    csvReader = csv.reader(csvfile, delimiter=',')
    for row in csvReader:
        test.append([row[2]])

test = test[1:]

#Word2index and Index2word

In [12]:
SOS_token = 0
EOS_token = 1

#helper class
class Lang:
  def __init__(self, name):
    self.name = name
    #word -> index
    self.word2index = {"SOS" : 0, "EOS" : 1, "UNK" : 2}
    #count of each word to later replace rare words
    self.word2count = {}
    #index -> word
    self.index2word = {0: "SOS", 1: "EOS", 2 : "UNK"}
    self.n_words = 3  # Count SOS, EOS, UNK tokens
    self.max_seq_length=-1

  def addSentence(self, sentence):
    #Find max seq length
    sentence_split = sentence.split(' ')
    if len(sentence_split) > self.max_seq_length:
      self.max_seq_length = len(sentence_split)
    for word in sentence.split(' '):
      self.addWord(word)

  def addWord(self, word):
    if word not in self.word2index:
      # word to index mapping
      self.word2index[word] = self.n_words
      self.word2count[word] = 1
      self.index2word[self.n_words] = word
      self.n_words += 1
    else:
      self.word2count[word] += 1

In [13]:
def clean_pairs(lines):
  cleaned = list()
  regex = re.compile('[%s]' % re.escape(string.punctuation))

  for pair in lines:
    clean_pair = list()
    
    #cleaning hindi phrases
    line = pair[0]
    line = normalizer.normalize(line)
    tokens = list()
    for t in indic_tokenize.trivial_tokenize(line): 
        tokens.append(t)
    line = tokens
    line = [word for word in line if not re.search(r'\d', word)]
    line = [regex.sub('', w) for w in line]
    #Replace the english characters
    line = [re.sub(r"[a-zA-Z]+?\s", "", w) for w in line]
    line = [re.sub(r'[-.।|,?;:<>&$₹]+','',w) for w in line]
    clean_pair.append(' '.join(line))

    #cleaning english phrases
    line = pair[1]
    line = normalize('NFD', line).encode('ascii', 'ignore')
    line = line.decode('UTF-8')
    line = word_tokenize(line) 
    line = [regex.sub('', w) for w in line]
    line = [re.sub(r"[^a-zA-Z0-9?.!,¿\-\/]+", " ", w) for w in line]
    line = [word for word in line if word.isalpha()]
    clean_pair.append(' '.join(line))
    
    cleaned.append(clean_pair)
  return array(cleaned)

#Creating the languages

In [14]:
def readLangs(lang1, lang2):
  #make Lang instances
  input_lang = Lang(lang1)
  output_lang = Lang(lang2)

  return input_lang, output_lang

##Since there are a lot of example sentences and we want to train something quickly, we’ll trim the data set to only relatively short and simple sentences. Here the maximum length is 30 words (that includes ending punctuation) 

In [15]:
MAX_LENGTH = 60

def filterPair(p):
  return len(p[0].split(' ')) <= MAX_LENGTH and len(p[1].split(' ')) <= MAX_LENGTH

def filterPairs(pairs):
  return [pair for pair in pairs if filterPair(pair)]

#Finally preparing the data.

In [29]:
def prepareData(lang1, lang2):
    input_lang, output_lang = readLangs(lang1, lang2)
    print("Read %s sentence pairs" % len(data))
    pairs = clean_pairs(data)
    pairs = filterPairs(pairs)

    print("Trimmed to %s sentence pairs" % len(pairs))
    #Make word lists from sentences in pairs
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    # for pair in pairst:
    #     input_lang.addSentence(pair[0])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs, test

#Loading the data and creating language objects

In [17]:
!head train1.csv

hindi,english
"एल सालवाडोर मे, जिन दोनो पक्षों ने सिविल-युद्ध से वापसी ली, उन्होंने वही काम किये जो कैदियों की कश्मकश के निदान हैं।","In El Salvador, both sides that withdrew from their civil war took moves that had been proven to mirror a prisoner's dilemma strategy."
मैं उनके साथ कोई लेना देना नहीं है.,I have nothing to do with them.
-हटाओ रिक.,"Fuck them, Rick."
क्योंकि यह एक खुशियों भरी फ़िल्म है.,Because it's a happy film.
The thought reaching the eyes...,The thought reaching the eyes...
मैंने तुमे School से हटवा दिया .,I got you suspended.
"यह Vika, एक फूल है.","It's a flower, Vika."
पर मेरे लिए उसका यहुदी विरोधी होना उसके कार्यों को और भी प्रशंसनीय बनाता है क्योंकि उसके पास भी पक्षपात करने के वही कारण थे जो बाकी फौजियों के पास थे पर उसकी सच जानने और उसे बनाए रखने की प्रेरणा सबसे ऊपर थी,"But personally, for me, the fact that Picquart was anti-Semitic actually makes his actions more admirable, because he had the same prejudices, the same reasons to be biased as his fellow officers

In [30]:
input_lang, output_lang, tr_pairs, pairst = prepareData('hindi', 'english')
print(random.choice(tr_pairs))

Read 101322 sentence pairs
Trimmed to 100926 sentence pairs
Counted words:
hindi 43447
english 29398
['आइवी लीग हाँ ' 'ivy league yeah']


In [25]:
print(random.choice(tr_pairs))

['बम फेंक कर तो आप मित्र नहीं बना सकते हैं न', 'you do not make friends by bombing them right']


In [31]:
from sklearn.model_selection import train_test_split

# split data into train and test set
pairs, val_pairs = train_test_split(tr_pairs, test_size=0.1, random_state = 12)

In [34]:
#Checking the pair of the test set size
print('Test set size is : ', len(test))

Test set size is :  5000


#Preparing the training data

###To train, for each pair we will need an input tensor (indexes of the words in the input sentence) and target tensor (indexes of the words in the target sentence). While creating these vectors we will append the EOS token to both sequences.

In [35]:
#Return a list of indexes, one for each word in the sentence
def indexesFromSentence(lang, sentence):
  indexes = []
  for word in sentence.split(' '):
    try:
      indexes.append(lang.word2index[word])
    except:
      indexes.append(lang.word2index["UNK"])
  return indexes

def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    #append the EOS token to both sequences - enables the model to define a distribution over sequences of all possible lengths.
    indexes.append(EOS_token)
    result = torch.LongTensor(indexes).view(-1,1).cuda()
    return result

def tensorsFromPair(pair):
    #input tensor - indexes of the words in the input sentence
    input_tensor = tensorFromSentence(input_lang, pair[0])
    #target tensor - indexes of the words in the target sentence
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [36]:
TARGET_MAX_LENGTH = output_lang.max_seq_length + 1 #since the we are adding the EOS and SOS tokens
print(TARGET_MAX_LENGTH)

INPUT_MAX_LENGTH = input_lang.max_seq_length + 1 #since the we are adding the EOS and SOS tokens
print(INPUT_MAX_LENGTH)

61
61


#Training the model

In [37]:
#Using teacher forcing causes it to converge faster
teacher_forcing_ratio = 0.55
#0.7
clip = 5.0

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, input_max_length=INPUT_MAX_LENGTH, target_max_length = TARGET_MAX_LENGTH):
    encoder_hidden = encoder.initHidden()
    # Zero gradients of both optimizers - Resetting
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    input_tensor = input_tensor.to(device)
    target_tensor = target_tensor.to(device)

    encoder_outputs = torch.zeros(input_max_length, encoder.hidden_size*2 , device=device) #Changed

    loss = 0

    #Pass the input sentence through the ENCODER and keep track of every output and the latest hidden state
    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    #decoder is given the <SOS> token as its first input
    decoder_input = torch.tensor([[SOS_token]], device=device)
    #decoder hidden state ("context vector" of input sentence ) ; last hidden state of the encoder as its first hidden state
    decoder_hidden = encoder_hidden

    #since using the "Teacher forcing" - passing the actual output sentence as the input to decoder
    if(random.random() < teacher_forcing_ratio):
      use_teacher_forcing = True
    else:
      use_teacher_forcing = False

    if use_teacher_forcing:
        #Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])     #For each decoder step (not the RNN step), compute the loss
            decoder_input = target_tensor[di]  # Teacher forcing via decoder_input

    else:
        #Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            
            #Get the index of best output word, and pass it as input to next decoder step
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    #backpropagate the loss 
    loss.backward()
    #Clipping the gradient after the backward pass
    torch.nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    torch.nn.utils.clip_grad_norm_(decoder.parameters(), clip)
    #apply the gradients using the optimization update step - update parameters
    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item()/target_length

In [38]:
import time
import math

def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [39]:
def trainIters(encoder, decoder, encoder_optimizer , decoder_optimizer, n_iters, print_every=10000, plot_every=10000):
    # Keep track of time elapsed and running averages
    start = time.time()
    train_plot_losses = []
    val_plot_losses = []
    train_plot_bleu_scores = []
    val_plot_bleu_scores = []
    max_val_bleu_score = -1.0

    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every
    print_bleu_total = 0

    training_pairs = []
    training_sentence_pairs = []

    # Get training data for this cycle
    for i in range(n_iters):
        sent_pair = random.choice(pairs)
        training_pairs.append(tensorsFromPair(sent_pair))
        #print(training_pairs)
        training_sentence_pairs.append(sent_pair)
    
    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]
        
        # Run the train function
        loss = train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, input_max_length = INPUT_MAX_LENGTH, target_max_length = TARGET_MAX_LENGTH )

        _, sent_bleu_score = evaluate_pairs(encoder , decoder, pairs = [training_sentence_pairs[iter-1]] )
        # Keep track of loss
        print_loss_total += loss
        plot_loss_total += loss
        print_bleu_total += sent_bleu_score

        #Compute the loss and Bleu score on both the training and development set
        if iter % print_every == 0:
            #print loss 
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) Training-set loss: %.4f' % (timeSince(start, iter / n_iters), iter, iter / n_iters * 100, print_loss_avg))

            #print BLEU score 
            train_bleu_score = print_bleu_total/print_every
            print_bleu_total = 0
            print('Training set Bleu score : %.4f ' %(train_bleu_score) )

            ### --------------  Development set -------------###
            val_loss , val_bleu_score = evaluate_pairs(encoder, decoder, val_pairs, show_translation = False)

            #print loss 
            print('%s (%d %d%%) validation loss : %.4f' % (timeSince(start, iter / n_iters),iter, iter / n_iters * 100, val_loss))
            
            #print BLEU score
            print('Validation set Bleu score : %.4f ' %(val_bleu_score) )

            train_plot_losses.append(print_loss_avg)
            val_plot_losses.append(val_loss)
            train_plot_bleu_scores.append(train_bleu_score)
            val_plot_bleu_scores.append(val_bleu_score)
            print('')

            if val_bleu_score >= max_val_bleu_score:
                print('Saving the Best Model!!!')
                max_val_bleu_score = val_bleu_score
                BEST_SAVE_PATH = './encoder_decoder_model_best_50.tar'
                torch.save({
                            'encoder_state_dict': encoder.state_dict(),
                            'attndecoder_state_dict': decoder.state_dict(),
                            'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),            
                            'decoder_optimizer_state_dict' : decoder_optimizer.state_dict(),
                            'criterion' : criterion.state_dict(),
                            }, BEST_SAVE_PATH)
    
    history = {'train_loss' : train_plot_losses,
               'val_loss': val_plot_losses,
               'train_bleu_score' : train_plot_bleu_scores,
               'val_bleu_score' : val_plot_bleu_scores
               }

    return history

#Building the Seq2Seq model



*   Encoder
*   Decoder






In [40]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, n_layers=1, bidirectional=False, verbose = False, batch_size=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.hidden_var = hidden_size // 2 if bidirectional else hidden_size
        self.n_layers = n_layers
        self.bidirectional = bidirectional
        self.n_directions = 2 if bidirectional else 1
        self.batch_size = batch_size
        self.verbose = verbose
        #An embedding to represent each word as a vector. Here our embedding length is 256
        self.embedding = nn.Embedding(input_size, hidden_size)

        # gru returns hidden state of all timesteps as well as hidden state at last timestep
        self.net = nn.GRU(hidden_size, hidden_size,num_layers=self.n_layers, bidirectional=self.bidirectional)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        #Forward propagate
        output, hidden = self.net(output, hidden)

        if self.verbose:
            print('Encoder input', input.shape)
            print('Encoder output', output.shape)
            print('Encoder hidden', hidden.shape)

        return output, hidden

    def initHidden(self):
        return torch.zeros(self.n_layers * self.n_directions, 1, self.hidden_size, device=device)

In [41]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [44]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, n_layers=1, dropout_p=0.1, max_length=INPUT_MAX_LENGTH, bidirectional=False):
        super(AttnDecoderRNN, self).__init__()

        #Define parameters
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        self.n_layers = n_layers
        self.bidirectional = bidirectional

        #Define layers
        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        #decoder’s input and hidden state as ip to attn layer
        self.attn = nn.Linear(self.hidden_size * 3, self.max_length)
        # self.attn = GeneralAttn(hidden_size)
        self.attn_combine = nn.Linear(self.hidden_size * 3, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)

        self.net = nn.GRU(self.hidden_size, self.hidden_size, num_layers=self.n_layers, bidirectional = self.bidirectional)
        self.out = nn.Linear(self.hidden_size*2, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        #Embed - Get the embedding of the current input word (last output word)
        embedded = self.embedding(input).view(1, 1, -1)   # S=1 x B x N
        embedded = self.dropout(embedded)
        self.hidden = hidden

        # Calculate attention weights and apply to encoder outputs 
        attn_weights = F.softmax(self.attn(torch.cat((embedded[0], hidden[0], hidden[1]), 1)), dim=1)

        attn_applied = torch.bmm(attn_weights.unsqueeze(0),encoder_outputs.unsqueeze(0))
        # Combine embedded input word and attended context, run through RNN
        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)
        output = F.relu(output)

        output, hidden = self.net(output, hidden)
        # Final output layer
        #Decode the hidden state of the last time step
        # Normalize attention to weights in range 0 to 1, resize to 1 x 1 x seq_len
        output = F.log_softmax(self.out(output[0]), dim=1)
        #Return final output, hidden state, and attention weights (for visualization)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(self.n_layers * self.n_directions, 1, self.hidden_size, device=device)

#Initializing the parameters

In [45]:
n_layers=2
bidirectional = True
hidden_size = 512
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, n_layers=n_layers, bidirectional=bidirectional).to(device)
attn_decoder1 = AttnDecoderRNN(hidden_size, output_lang.n_words, n_layers=n_layers, bidirectional=bidirectional).to(device)

In [ ]:
attn_decoder1

In [ ]:
encoder1

In [46]:
#torch.no_grad removes the ability to perform backprop (that's fine as we're 'evaluating' our model)
#but speeds up computation and reduces RAM consumption
def evaluate(encoder, decoder, pair, input_max_length = INPUT_MAX_LENGTH, target_max_length = TARGET_MAX_LENGTH):
    with torch.no_grad():

        #Prepare input and output tensors from the sentences
        input_tensor = tensorFromSentence(input_lang, pair[0])
        target_tensor = tensorFromSentence(output_lang, pair[1])
        input_length = input_tensor.size()[0]
        target_length = target_tensor.size()[0]

        input_tensor = input_tensor.to(device)
        target_tensor = target_tensor.to(device)

        #Run through encoder
        #Generate the output sentence 
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(input_max_length, encoder.hidden_size*2, device=device) ## Changed

        loss = 0

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0] #BUG '+'

        #Create starting vectors for decoder
        decoder_input = torch.tensor([[SOS_token]], device=device)  #SOS
        decoder_hidden = encoder_hidden
        decoded_words = []
        decoder_attentions = torch.zeros(input_max_length, input_max_length)

        #Run through decoder
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
            #store the decoder’s attention outputs
            decoder_attentions[di] = decoder_attention.data
            #Choose top word from output
            topv, topi = decoder_output.data.topk(1)

            loss += criterion(decoder_output, target_tensor[di]).item()

            #if prediction is EOS, STOP!!!
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])   #else add to output string 

            #feed the decoder’s predictions back to itself  - # Next input is chosen word
            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions, loss/target_length

In [47]:
learning_rate=0.001
encoder_optimizer = optim.Adam(encoder1.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(attn_decoder1.parameters(), lr=learning_rate)

In [ ]:
encoder_optimizer

In [48]:
# criterion = nn.NLLLoss()
criterion = nn.CrossEntropyLoss()

In [49]:
SAVE_PATH = './encoder_decoder_model_50.tar'

# #### General checkpoint saving for inference or resuming training
torch.save({
            'encoder_state_dict': encoder1.state_dict(),
            'attndecoder_state_dict': attn_decoder1.state_dict(),
            'encoder_optimizer_state_dict': encoder_optimizer.state_dict(),            
            'decoder_optimizer_state_dict' : decoder_optimizer.state_dict(),
            'criterion' : criterion.state_dict(),
            }, SAVE_PATH)

#Importing for Bleu score

In [50]:
from nltk.translate.bleu_score import sentence_bleu
from nltk.translate.bleu_score import SmoothingFunction

In [51]:
def evaluate_pairs(encoder, decoder, pairs, show_translation = False):
    n = len(pairs)
    #Compute the average bleu score 

    bleu_scores_list = []
    loss_list = []

    for i in range(n):
        # pair = random.choice(pairs)
        pair = pairs[i]
        
        output_words, attentions, loss_value  = evaluate(encoder, decoder, pair, input_max_length = INPUT_MAX_LENGTH, target_max_length = TARGET_MAX_LENGTH )
        output_sentence = ' '.join(output_words)

        # f1.writelines(pair[0]+","+output_sentence+"\n")

        if show_translation == True:
            print('Hindi sentence : ', pair[0])
            print('English sentence : ', pair[1])
            print('Predicted sentence : ', output_sentence)

        actual_sent_splitted = pair[1].split()
        output_sent_splitted = output_sentence.split()
        
        smoothing_function = SmoothingFunction().method4
        sent_bleu_score = 0.0

        try:
          sent_bleu_score = sentence_bleu(references = [actual_sent_splitted] , hypothesis = output_sent_splitted , weights = (0.25, 0.25, 0.25, 0.25), smoothing_function =  smoothing_function )
        except:
          pass

        bleu_scores_list.append(sent_bleu_score)
        loss_list.append(loss_value)

    avg_bleu_score = np.nanmean(bleu_scores_list, )
    avg_loss_value = np.nanmean(loss_list)
    # print('Bleu score is : ', avg_bleu_score )
    # print('')
    return avg_loss_value, avg_bleu_score

In [52]:
%matplotlib inline

training_history = trainIters(encoder = encoder1, decoder = attn_decoder1, 
                              encoder_optimizer = encoder_optimizer, decoder_optimizer = decoder_optimizer,
                              n_iters = 30000, print_every=1000)

1m 34s (- 45m 49s) (1000 3%) Training-set loss: 6.1500
Training set Bleu score : 0.0614 
4m 44s (- 137m 21s) (1000 3%) validation loss : 5.4866
Validation set Bleu score : 0.0703 

Saving the Best Model!!!
6m 35s (- 92m 17s) (2000 6%) Training-set loss: 5.9318
Training set Bleu score : 0.0975 
9m 34s (- 134m 3s) (2000 6%) validation loss : 4.8861
Validation set Bleu score : 0.0755 

Saving the Best Model!!!
11m 14s (- 101m 6s) (3000 10%) Training-set loss: 5.8477
Training set Bleu score : 0.1136 
13m 56s (- 125m 25s) (3000 10%) validation loss : 4.2079
Validation set Bleu score : 0.0779 

Saving the Best Model!!!
15m 38s (- 101m 42s) (4000 13%) Training-set loss: 5.6945
Training set Bleu score : 0.1152 
19m 0s (- 123m 36s) (4000 13%) validation loss : 5.8872
Validation set Bleu score : 0.1170 

Saving the Best Model!!!
20m 39s (- 103m 15s) (5000 16%) Training-set loss: 5.6021
Training set Bleu score : 0.1188 
24m 3s (- 120m 16s) (5000 16%) validation loss : 5.9291
Validation set Bleu s

In [ ]:
# device = torch.device('cpu')
encoder_loaded = EncoderRNN(input_lang.n_words, hidden_size, n_layers=n_layers,  method='GRU', bidirectional=bidirectional)
attn_decoder_loaded= AttnDecoderRNN(hidden_size, output_lang.n_words, n_layers=n_layers,method='GRU', bidirectional=bidirectional)
#Load the dictionary from saved file
BEST_SAVE_PATH = './encoder_decoder_model_best_50.tar'
checkpoint = torch.load(BEST_SAVE_PATH)

# load the state from dictionary 
encoder_loaded.load_state_dict(checkpoint['encoder_state_dict'])
attn_decoder_loaded.load_state_dict(checkpoint['attndecoder_state_dict'])

encoder_loaded.train()
attn_decoder_loaded.train()

encoder_loaded = encoder_loaded.to(device)
attn_decoder_loaded = attn_decoder_loaded.to(device)

learning_rate=0.001
encoder_optimizer_loaded = optim.Adam(encoder_loaded.parameters(), lr=learning_rate)
decoder_optimizer_loaded = optim.Adam(attn_decoder_loaded.parameters(), lr=learning_rate)

In [ ]:
def evaluate1(encoder, decoder, sentence, max_length=INPUT_MAX_LENGTH,  target_max_length = TARGET_MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
#         input_tensor = input_tensor.to(device)
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size*2, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei], encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(target_max_length, target_max_length)

        for di in range(target_max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(decoder_input, decoder_hidden, encoder_outputs)
#             decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()
        #print(decoded_words)
        return decoded_words

In [ ]:
f1= open("answer.txt",'w')

In [ ]:
def evaluateRandomly(encoder, decoder, pairs1):
    for i in range(len(pairs1)):
        pair = pairs1[i]
        # pair = random.choice(pairs)
        #print('>',pair[0])
        output_words = evaluate1(encoder, decoder,pair[0])
        output_sentence = ' '.join(output_words[:-1])
        f1.writelines(output_sentence+"\n")
        #print('<', output_sentence)
        #print('')

In [ ]:
evaluateRandomly(encoder1, attn_decoder1, pairst)

In [ ]:
pairst[:2]

In [ ]:
test_loss , test_bleu_score = evaluate_pairs(encoder = encoder1, decoder = attn_decoder1, pairs = pairst[5:15], show_translation = True)

In [ ]:
f1.close()

In [ ]:
SAVE_PATH = './encoder_decoder_model_best_50.tar'

In [ ]:
# ##  Load the dictionary from saved file
checkpoint = torch.load(SAVE_PATH)

# load the state from dictionary 
encoder1.load_state_dict(checkpoint['encoder_state_dict'])
attn_decoder1.load_state_dict(checkpoint['attndecoder_state_dict'])

In [ ]:
encoder1.train().to(device)
attn_decoder1.train().to(device)